In [ ]:
import pickle
import numpy as np
from PIL import Image
import csv
import os

In [ ]:
with open('datasets/colorferet_sketch_to_img_map.pickle', 'rb') as f:
    colorferet_map = pickle.load(f)

In [ ]:
i = 0
for k, v in colorferet_map.items():
    print(k, v)
    i += 1
    if i > 5:
        break

In [ ]:
# Testing that you can transform one triangle into another
fiducial = np.array([[312, 450, 387], 
                     [348, 352, 497]])
fiducial_3d = np.array([[312, 450, 387], 
                        [348, 352, 497], 
                        [1, 1, 1]])
goal = np.array([[300, 420, 360], 
                 [350, 250, 500], 
                 [0, 0, 0]])
w = goal @ np.linalg.inv(fiducial_3d)
mat = w[:2, :2]
translation = w[:2, 2:3]
mat @ fiducial + translation

In [ ]:
# Expects fiducial_points is 3 x 2 numpy array
def align_face(img, fiducial_points, out_width, out_height, point_targets=None):
    orig_width, orig_height = img.size
    if point_targets is None:
        point_targets = np.array([[0.375, 0.625, 0.5], [0.5, 0.5, 0.72]]) * np.array([[out_width], [out_height]])
    # Pupil 1: (0.4, 0.5), Pupil 2: (0.6, 0.5), Mouth: (0.5, 0.7)
    
    fiducial_3d = np.vstack((fiducial_points.T, np.ones((1, 3))))
    transform = np.vstack((point_targets @ np.linalg.inv(fiducial_3d), np.array([0, 0, 1])))
    transform_inv = np.linalg.inv(transform)

    data = transform_inv.flatten()[:6]
    return img.transform((out_width, out_height), Image.AFFINE, data)

In [ ]:
out_width = 200
out_height = 200
square_crop_point_targets = np.array([[0.375, 0.625, 0.5], [0.375, 0.375, 0.65]]) * np.array([[out_width], [out_height]]) # meant to be with output 200 x 250
# mega_crop_point_targets = np.array([[0.33, 0.67, 0.5], [0.48, 0.48, 0.83]]) * np.array([[out_width], [out_height]]) # meant to be with square output
hyper_crop_point_targets = np.array([[0.31, 0.69, 0.5], [0.5, 0.5, 0.9]]) * np.array([[out_width], [out_height]]) # meant to be with square output

In [ ]:
img1 = Image.open('datasets/colorferet/original_sketch/00001.jpg')
fiducial_points = np.zeros((3, 2))
with open('datasets/colorferet/sketch_points/00001.3pts', 'r') as f:
    reader = csv.reader(f, delimiter= ' ')
    for i, row in enumerate(reader):
        fiducial_points[i] = row
aligned1 = align_face(img1, fiducial_points, out_width, out_height, hyper_crop_point_targets)
aligned1

In [ ]:
img = Image.open('datasets/colorferet/color/01006_960627_fa.ppm')
fiducial_points = np.zeros((3, 2))
with open('datasets/colorferet/photo_points/01006fa010_960627.3pts', 'r') as f:
    reader = csv.reader(f, delimiter= ' ')
    for i, row in enumerate(reader):
        fiducial_points[i] = row
fiducial_points += np.array([[-100, 0], [-100, 0], [-100, 0]])
aligned = align_face(img, fiducial_points, out_width, out_height, square_crop_point_targets)
aligned

In [ ]:
aligned1 = align_face(img1, fiducial_points, out_width=200, out_height=250)
aligned1.save('datasets/colorferet/aligned_sketches/00001.jpg', 'JPEG')


In [ ]:
# duplicates
unmapped_duplicates = [
    '00615_941031_fa.ppm',
    '00185_940422_fb.ppm',
    '00621_941121_fb.ppm',
    '00210_940422_fb.ppm',
    '00256_940128_fa.ppm',
    '00498_960627_fa.ppm',
    '00012_930831_fa_a.ppm', # w funny sunglasses
    '00283_940307_fa.ppm',
    '00531_960627_fa.ppm',
    '00807_960530_fa.ppm',
    '00184_940422_fa.ppm'
]

typo_color_to_points = {
    '00429_940422_fa.ppm': '00429fa011_940422.3pts',
    '00086_940422_fa.ppm': '00086fa010_930422.3pts', 
    '00450_940422_fa.ppm': '00450fa011_940422.3pts',
    '00436_940422_fa.ppm': '00436fa011_940422.3pts', 
    '00378_940422_fa.ppm': '00378fa011_940422.3pts',
    '00442_940422_fa.ppm': '00442fa011_940422.3pts',
    '00095_940128_fa.ppm': '00095fa010_930128.3pts',
    '00458_940422_fa.ppm': '00458fa011_940422.3pts',
    '01208_94012some8_fa_a.ppm': '01208fa010_960627.3pts'
}

In [ ]:
def align_all_in_folder(src_dir, points_dir, dest_dir, out_width, out_height, point_targets, src_name_to_pts_name, src_to_pts_special, unmapped_duplicates, preprocess_fiducials=None, fixed_fiducials=None):
    for root, _, fnames in sorted(os.walk(src_dir)):
        for fname in fnames:
            if fname in unmapped_duplicates:
                continue
            path = os.path.join(root, fname)
            try:
                with open(path, 'rb') as image_file:
                    img = Image.open(path)
            except Exception as err:
                print(fname, err)
                continue
            with open(path, 'rb') as image_file:
                img = Image.open(path)
                fiducial_points = np.zeros((3, 2))
                if fixed_fiducials is not None:
                    fiducial_points = fixed_fiducials
                else:
                    if fname in src_to_pts_special:
                        points_fname = src_to_pts_special[fname]
                    else:
                        points_fname = src_name_to_pts_name(fname)
                    points_path = os.path.join(points_dir, points_fname)
                    try:
                        with open(points_path, 'r') as f:
                            reader = csv.reader(f, delimiter= ' ')
                            for i, row in enumerate(reader):
                                fiducial_points[i] = row
                    except OSError as err:
                        print(fname, err)
                        continue
                    if preprocess_fiducials is not None:
                        fiducial_points = preprocess_fiducials(fiducial_points)
                aligned = align_face(img, fiducial_points, out_width, out_height, point_targets)
                out_fname_jpg = fname[:-3] + 'jpg'
                out_path = os.path.join(dest_dir, out_fname_jpg)
                with open(out_path, 'wb') as out_file:
                    aligned.save(out_file, 'JPEG')

In [ ]:
align_all_in_folder('datasets/colorferet/original_sketch', 
                    'datasets/colorferet/sketch_points',
                    'datasets/colorferet/supercropped_sketches',
                    out_width,
                    out_heightloa,
                    hyper_crop_point_targets,
                    lambda name: name[:-4] + '.3pts',
                    {},
                    [])


In [ ]:
align_all_in_folder('datasets/colorferet/original_sketch', 
                    'datasets/colorferet/sketch_points',
                    'datasets/colorferet/cropped_sketches',
                    out_width,
                    out_height,
                    square_crop_point_targets,
                    lambda name: name[:-4] + '.3pts',
                    {},
                    [])

In [ ]:
align_all_in_folder('datasets/colorferet/color', 
                    'datasets/colorferet/photo_points',
                    'datasets/colorferet/supercropped_color',
                    out_width,
                    out_height,
                    hyper_crop_point_targets,
                    lambda src: src[:5] + src[13:15] + '010_' + src[6:12] + '.3pts',
                    typo_color_to_points,
                    unmapped_duplicates,
                    lambda fiducials: fiducials + np.array([[-100, 0], [-100, 0], [-100, 0]]))

In [ ]:
align_all_in_folder('datasets/colorferet/color', 
                    'datasets/colorferet/photo_points',
                    'datasets/colorferet/cropped_color',
                    out_width,
                    out_height,
                    square_crop_point_targets,
                    lambda src: src[:5] + src[13:15] + '010_' + src[6:12] + '.3pts',
                    typo_color_to_points,
                    unmapped_duplicates,
                    lambda fiducials: fiducials + np.array([[-100, 0], [-100, 0], [-100, 0]]))

In [ ]:
cuhk_fiducials = np.array([[0.375, 0.625, 0.5], [0.5, 0.5, 0.74]]) * np.array([[200], [250]]) # meant to be with output 200 x 250
cuhk_fiducials = cuhk_fiducials.T

In [ ]:
imgs_and_save_paths = align_all_in_folder('datasets/CUHK/train', 
                    'doesnt-matter',
                    'datasets/CUHK/cropped',
                    out_width,
                    out_height,
                    square_crop_point_targets,
                    None,
                    None,
                    [],
                    None,
                    cuhk_fiducials)

In [ ]:
imgs_and_save_paths = align_all_in_folder('datasets/CUHK/train', 
                    'doesnt-matter',
                    'datasets/CUHK/supercropped',
                    out_width,
                    out_height,
                    hyper_crop_point_targets,
                    None,
                    None,
                    [],
                    None,
                    cuhk_fiducials)